In [1]:
 import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, normalize
from sklearn import metrics
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import warnings
import pickle

sys.path.append(os.path.abspath(os.path.join('../scripts')))

from clean_tellco_dataframe import Tellco
from plot_dataframe import Plotter

plot = Plotter()
cleaner = Tellco()

In [2]:
warnings.filterwarnings('ignore')

In [3]:
tellco_df = pd.read_csv("../data/clean_data.csv")

In [4]:
tellco_exprience_df = tellco_df[['MSISDN/Number','Avg RTT DL (ms)','Avg RTT UL (ms)','Avg Bearer TP DL (kbps)','Avg Bearer TP UL (kbps)','TCP DL Retrans. Vol (Bytes)','TCP UL Retrans. Vol (Bytes)','Handset Type']]
tellco_exprience_df

,MSISDN/Number,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Handset Type
0,33664962239,42.000000,5.000000,23.0,44.0,2.080991e+07,759658.664811,Samsung Galaxy A5 Sm-A520F
1,33681854413,65.000000,5.000000,16.0,26.0,2.080991e+07,759658.664811,Samsung Galaxy J5 (Sm-J530)
2,33760627129,109.795706,17.662883,6.0,9.0,2.080991e+07,759658.664811,Samsung Galaxy A8 (2018)
3,33699795932,109.795706,17.662883,6.0,9.0,2.080991e+07,759658.664811,Samsung Sm-G390F
4,33665368271,102.000000,5.000000,22.0,38.0,2.080991e+07,759658.664811,Huawei Mate 20 Lite
...,...,...,...,...,...,...,...,...
138066,33645655643,109.795706,17.662883,55.0,54.0,2.080991e+07,759658.664811,Samsung Galaxy J5 (Sm-J530)
138067,33650688697,32.000000,0.000000,52.0,65.0,2.080991e+07,759658.664811,Apple iPhone 8 Plus (A1897)
138068,33663449963,27.000000,2.000000,23.0,54.0,2.080991e+07,759658.664811,Apple iPhone Se (A1723)
138069,33621890103,43.000000,6.000000,43.0,47.0,2.080991e+07,759658.664811,Apple iPhone Xs (A2097)


In [5]:
tellco_exprience_df['Total Avg RTT (ms)'] = tellco_exprience_df['Avg RTT DL (ms)'] + tellco_exprience_df['Avg RTT UL (ms)']
tellco_exprience_df['Total Avg Bearer TP (kbps)'] = tellco_exprience_df['Avg Bearer TP DL (kbps)'] + tellco_exprience_df['Avg Bearer TP UL (kbps)']
tellco_exprience_df['Total TCP Retrans. Vol (Bytes)'] = tellco_exprience_df['TCP DL Retrans. Vol (Bytes)'] + tellco_exprience_df['TCP UL Retrans. Vol (Bytes)']
tellco_exprience_df.head()

,MSISDN/Number,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Handset Type,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
0,33664962239,42.000000,5.000000,23.0,44.0,2.080991e+07,759658.664811,Samsung Galaxy A5 Sm-A520F,47.000000,67.0,2.156957e+07
1,33681854413,65.000000,5.000000,16.0,26.0,2.080991e+07,759658.664811,Samsung Galaxy J5 (Sm-J530),70.000000,42.0,2.156957e+07
2,33760627129,109.795706,17.662883,6.0,9.0,2.080991e+07,759658.664811,Samsung Galaxy A8 (2018),127.458589,15.0,2.156957e+07
3,33699795932,109.795706,17.662883,6.0,9.0,2.080991e+07,759658.664811,Samsung Sm-G390F,127.458589,15.0,2.156957e+07
4,33665368271,102.000000,5.000000,22.0,38.0,2.080991e+07,759658.664811,Huawei Mate 20 Lite,107.000000,60.0,2.156957e+07


In [6]:
tellco_exprience_df = tellco_exprience_df[['MSISDN/Number','Total Avg RTT (ms)','Total Avg Bearer TP (kbps)','Total TCP Retrans. Vol (Bytes)','Handset Type']]
tellco_exprience_df.head()

,MSISDN/Number,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
0,33664962239,47.000000,67.0,2.156957e+07,Samsung Galaxy A5 Sm-A520F
1,33681854413,70.000000,42.0,2.156957e+07,Samsung Galaxy J5 (Sm-J530)
2,33760627129,127.458589,15.0,2.156957e+07,Samsung Galaxy A8 (2018)
3,33699795932,127.458589,15.0,2.156957e+07,Samsung Sm-G390F
4,33665368271,107.000000,60.0,2.156957e+07,Huawei Mate 20 Lite


In [7]:
# tellco_exprience_df = tellco_exprience_df.copy()
tellco_exprience_df1 = tellco_exprience_df.groupby(
    'MSISDN/Number').agg({'Total Avg RTT (ms)': 'sum', 'Total Avg Bearer TP (kbps)': 'sum', 'Total TCP Retrans. Vol (Bytes)': 'sum','Handset Type': [lambda x: x.mode()[0]] }) #' '.join(x)
tellco_exprience_df1

,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
,sum,sum,sum,<lambda>
MSISDN/Number,,,,
33601001722,46.000000,76.0,2.156957e+07,Huawei P20 Lite Huawei Nova 3E
33601001754,31.000000,99.0,2.156957e+07,Apple iPhone 7 (A1778)
33601007832,84.000000,248.0,7.607247e+05,Apple iPhone 5S (A1457)
33601008617,119.000000,43342.0,3.094040e+07,Apple iPhone Se (A1723)
33601010682,151.458589,7908.0,2.233199e+07,Samsung Galaxy A8 (2018)
...,...,...,...,...
33789960306,127.458589,106.0,2.156957e+07,Samsung Galaxy S8 Plus (Sm-G955F)
33789967113,53.000000,67.0,2.156957e+07,Apple iPhone 6S (A1688)


In [8]:
tellco_exprience_df = pd.DataFrame(columns=[
    "Total Avg RTT (ms)",
    "Total Avg Bearer TP (kbps)",
    "Total TCP Retrans. Vol (Bytes)",
    "Handset Type"])

tellco_exprience_df["Total Avg RTT (ms)"] = tellco_exprience_df1["Total Avg RTT (ms)"]['sum']
tellco_exprience_df["Total Avg Bearer TP (kbps)"] = tellco_exprience_df1["Total Avg Bearer TP (kbps)"]['sum']
tellco_exprience_df["Total TCP Retrans. Vol (Bytes)"] = tellco_exprience_df1["Total TCP Retrans. Vol (Bytes)"]['sum']
tellco_exprience_df["Handset Type"] = tellco_exprience_df1["Handset Type"]['<lambda>']
tellco_exprience_df.head()

,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
MSISDN/Number,,,,
33601001722,46.000000,76.0,2.156957e+07,Huawei P20 Lite Huawei Nova 3E
33601001754,31.000000,99.0,2.156957e+07,Apple iPhone 7 (A1778)
33601007832,84.000000,248.0,7.607247e+05,Apple iPhone 5S (A1457)
33601008617,119.000000,43342.0,3.094040e+07,Apple iPhone Se (A1723)
33601010682,151.458589,7908.0,2.233199e+07,Samsung Galaxy A8 (2018)


In [9]:
tp = tellco_exprience_df.sort_values('Total Avg Bearer TP (kbps)', ascending=False)
tp.head(10)['Total Avg Bearer TP (kbps)']

MSISDN/Number
33762333464    460977.0
33659546392    436825.0
33614892860    384723.0
33699248832    330360.0
33662780927    329222.0
33699327554    326999.0
33604515716    324108.0
33658946065    322620.0
33658263267    317825.0
33661295438    314912.0
Name: Total Avg Bearer TP (kbps), dtype: float64

In [10]:
tp.tail(10)['Total Avg Bearer TP (kbps)']

MSISDN/Number
33634247429    0.0
33604131582    0.0
33662203388    0.0
33662540046    0.0
33661089447    0.0
33659471143    0.0
33667414355    0.0
33761309821    0.0
33616110252    0.0
33658211925    0.0
Name: Total Avg Bearer TP (kbps), dtype: float64

In [11]:
tellco_exprience_df['Total Avg Bearer TP (kbps)'].value_counts().head(10)

55060.0    2358
63.0       2017
15.0       1972
97.0        945
90.0        939
98.0        889
96.0        803
99.0        791
89.0        770
93.0        759
Name: Total Avg Bearer TP (kbps), dtype: int64

In [12]:
br_tp = tellco_exprience_df.sort_values('Total Avg Bearer TP (kbps)', ascending=False)
br_tp

,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
MSISDN/Number,,,,
33762333464,400.000000,460977.0,2.788275e+07,Samsung Galaxy A8 (2018)
33659546392,588.000000,436825.0,3.472787e+07,Huawei B528S-23A
33614892860,1388.408741,384723.0,1.344178e+08,Samsung Galaxy J5 (Sm-J530)
33699248832,549.000000,330360.0,1.717218e+08,Huawei B528S-23A
33662780927,574.971622,329222.0,8.699571e+06,Huawei B528S-23A
...,...,...,...,...
33659471143,127.458589,0.0,2.156957e+07,Huawei P20 Pro
33667414355,127.458589,0.0,2.156957e+07,Samsung Galaxy S (Gt-I9000)
33761309821,141.795706,0.0,2.156957e+07,Lg Lg-T385


In [13]:
br_tp.head(10)['Total Avg Bearer TP (kbps)']

MSISDN/Number
33762333464    460977.0
33659546392    436825.0
33614892860    384723.0
33699248832    330360.0
33662780927    329222.0
33699327554    326999.0
33604515716    324108.0
33658946065    322620.0
33658263267    317825.0
33661295438    314912.0
Name: Total Avg Bearer TP (kbps), dtype: float64

In [14]:
br_tp.tail(10)['Total Avg Bearer TP (kbps)']

MSISDN/Number
33634247429    0.0
33604131582    0.0
33662203388    0.0
33662540046    0.0
33661089447    0.0
33659471143    0.0
33667414355    0.0
33761309821    0.0
33616110252    0.0
33658211925    0.0
Name: Total Avg Bearer TP (kbps), dtype: float64

In [15]:
tellco_exprience_df['Total Avg Bearer TP (kbps)'].value_counts().head(10)

55060.0    2358
63.0       2017
15.0       1972
97.0        945
90.0        939
98.0        889
96.0        803
99.0        791
89.0        770
93.0        759
Name: Total Avg Bearer TP (kbps), dtype: int64

In [16]:
tcp_retransmitted = tellco_exprience_df.sort_values('Total TCP Retrans. Vol (Bytes)', ascending=False)
tcp_retransmitted

,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
MSISDN/Number,,,,
33658162536,744.000000,243138.0,3.006502e+08,Huawei B528S-23A
33663654533,700.971622,229214.0,3.004731e+08,Huawei B528S-23A
33763730582,596.000000,275208.0,2.890404e+08,Huawei B528S-23A
33699231421,1445.921774,133805.0,2.844869e+08,Huawei B528S-23A
33665639832,803.000000,198587.0,2.715699e+08,Huawei B528S-23A
...,...,...,...,...
33607833304,32.000000,16853.0,1.430000e+02,Apple iPhone 5C (A1529)
33626357513,40.000000,10964.0,1.340000e+02,Apple iPhone 7 (A1778)
33762062356,41.000000,32577.0,1.290000e+02,Apple iPhone 8 (A1905)


In [17]:
tcp_retransmitted.head(10)['Total TCP Retrans. Vol (Bytes)']

MSISDN/Number
33658162536    3.006502e+08
33663654533    3.004731e+08
33763730582    2.890404e+08
33699231421    2.844869e+08
33665639832    2.715699e+08
33625779332    2.621829e+08
33669105695    2.588681e+08
33668144136    2.569291e+08
33667806184    2.530505e+08
33762644658    2.523010e+08
Name: Total TCP Retrans. Vol (Bytes), dtype: float64

In [18]:
tcp_retransmitted.tail(10)['Total TCP Retrans. Vol (Bytes)']

MSISDN/Number
33699694715    182.0
33679366089    179.0
33784203839    177.0
33769636458    176.0
33650681239    176.0
33607833304    143.0
33626357513    134.0
33762062356    129.0
33614777138    128.0
33650128412     97.0
Name: Total TCP Retrans. Vol (Bytes), dtype: float64

In [19]:
tellco_exprience_df['Total TCP Retrans. Vol (Bytes)'].value_counts().head(10)

2.156957e+07    42973
4.313915e+07     6850
6.470872e+07      948
2.081121e+07      356
8.627829e+07      229
2.081124e+07      150
7.609887e+05      141
5.187239e+07      137
4.238078e+07       81
2.081123e+07       78
Name: Total TCP Retrans. Vol (Bytes), dtype: int64

In [20]:
tellco_exprience_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99115 entries, 33601001722 to 337000037000919
Data columns (total 4 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Total Avg RTT (ms)              99115 non-null  float64
 1   Total Avg Bearer TP (kbps)      99115 non-null  float64
 2   Total TCP Retrans. Vol (Bytes)  99115 non-null  float64
 3   Handset Type                    99115 non-null  object 
dtypes: float64(3), object(1)
memory usage: 3.8+ MB


In [21]:
handset_type_exprience_df = tellco_exprience_df.groupby('Handset Type').agg({'Total Avg Bearer TP (kbps)': 'mean', 'Total TCP Retrans. Vol (Bytes)': 'mean','Total Avg RTT (ms)': "mean"}) #.mean()
handset_type_exprience_df #.sort_values(by='Total Avg Bearer TP (kbps)', ascending=False).head()

,Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Total Avg RTT (ms)
Handset Type,,,
A-Link Telecom I. Cubot A5,23510.0,2.156957e+07,42.000000
A-Link Telecom I. Cubot Note Plus,6699.0,6.162130e+05,231.489265
A-Link Telecom I. Cubot Note S,8937.0,4.141173e+07,227.489265
A-Link Telecom I. Cubot Nova,52703.0,1.371700e+05,44.000000
A-Link Telecom I. Cubot Power,52495.0,8.041000e+03,43.000000
...,...,...,...
Zte Racer Iii Mini Zte Switch X1 Bouygues Telecom Bs 402 Blade Q Mini Zte Blade G Pro,1549.0,8.789197e+06,183.697853
Zte Zte Blade C2 Smartphone Android By Sfr Startrail 4 Zte Blade Flex T809 Zte T809,58.0,1.116595e+07,165.729294
Zyxel Communicat. Lte7460,42648.0,2.565253e+07,126.657207


In [22]:
handset_type_exprience_df.sort_values('Total TCP Retrans. Vol (Bytes)', ascending=False).head()

,Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Total Avg RTT (ms)
Handset Type,,,
Huawei Me909S-120,1548.5,7.986013e+07,410.161810
Sierra Wireless Usb305,31715.0,7.311910e+07,249.000000
Apple iPhone Xr (A2106),12864.5,5.396322e+07,195.000000
Zyxel Communicat. Sbg3600,55060.0,5.187239e+07,131.657207
Huawei E587 E587U-2 Turkcell Vinnwifi E587,22452.0,5.071530e+07,232.000000


In [23]:
tellco_exprience_df.drop(['Handset Type'], axis=1, inplace=True)

In [24]:

scale_data = StandardScaler().fit_transform(tellco_exprience_df)
scale_data

array([[-0.67078091, -0.55096718,  0.0441717 ],
       [-0.81381178, -0.55023357,  0.0441717 ],
       [-0.30843605, -0.54548107, -1.12899202],
       ...,
       [-0.61356857, -0.22278955, -1.15958728],
       [-0.72799326, -0.54969134,  0.0441717 ],
       [ 0.10595858, -0.5523387 ,  0.0441717 ]])

In [25]:
normalized_data = normalize(scale_data)
normalized_data

array([[-0.77174548, -0.63389763,  0.05082033],
       [-0.82758239, -0.55954415,  0.04491913],
       [-0.23886777, -0.42244688, -0.87434594],
       ...,
       [-0.46108945, -0.16742369, -0.87141599],
       [-0.79711681, -0.60188498,  0.04836584],
       [ 0.18782248, -0.97907712,  0.07829887]])

In [26]:
kmeans = KMeans(n_clusters = 3, random_state = 42).fit(normalized_data)
y_kmeans = kmeans.fit_predict(normalized_data)
X = np.array(normalized_data)
y_kmeans

array([0, 0, 2, ..., 2, 0, 0])

In [27]:
clustered_tellco_exprience_df = tellco_exprience_df.copy()
clustered_tellco_exprience_df.insert(0, 'Cluster', y_kmeans)
clustered_tellco_exprience_df

,Cluster,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
MSISDN/Number,,,,
33601001722,0,46.000000,76.0,2.156957e+07
33601001754,0,31.000000,99.0,2.156957e+07
33601007832,2,84.000000,248.0,7.607247e+05
33601008617,1,119.000000,43342.0,3.094040e+07
33601010682,1,151.458589,7908.0,2.233199e+07
...,...,...,...,...
33789960306,0,127.458589,106.0,2.156957e+07
33789967113,0,53.000000,67.0,2.156957e+07
33789996170,2,52.000000,10365.0,2.180450e+05


In [28]:
clustered_tellco_exprience_df[clustered_tellco_exprience_df["Cluster"]==0][["Total Avg RTT (ms)","Total Avg Bearer TP (kbps)","Total TCP Retrans. Vol (Bytes)"]].describe()

,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
count,49674.000000,49674.000000,4.967400e+04
mean,70.859446,726.335447,2.342172e+07
std,40.683601,2765.459526,6.101414e+06
min,0.000000,0.000000,7.801046e+06
25%,36.000000,78.000000,2.156957e+07
50%,57.000000,99.000000,2.156957e+07
75%,127.458589,128.000000,2.156957e+07
max,169.000000,27465.000000,6.470872e+07


In [29]:
clustered_tellco_exprience_df[clustered_tellco_exprience_df["Cluster"]==1][["Total Avg RTT (ms)","Total Avg Bearer TP (kbps)","Total TCP Retrans. Vol (Bytes)"]].describe()

,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
count,22673.000000,22673.00000,2.267300e+04
mean,227.233414,43873.85948,3.696848e+07
std,147.901021,48471.15317,2.430222e+07
min,27.000000,1.00000,1.032700e+04
25%,128.000000,3098.00000,2.177898e+07
50%,195.458589,35376.00000,2.823077e+07
75%,262.000000,55150.00000,4.390334e+07
max,1938.732362,460977.00000,3.006502e+08


In [30]:
clustered_tellco_exprience_df[clustered_tellco_exprience_df["Cluster"]==2][["Total Avg RTT (ms)","Total Avg Bearer TP (kbps)","Total TCP Retrans. Vol (Bytes)"]].describe()

,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
count,26768.000000,26768.000000,2.676800e+04
mean,106.834397,25732.272340,2.188278e+06
std,69.136901,21365.675667,3.879642e+06
min,16.000000,8.000000,9.700000e+01
25%,52.000000,6618.250000,1.842602e+05
50%,80.657207,21901.000000,7.663007e+05
75%,148.657207,42309.500000,2.126774e+06
max,360.489265,143740.000000,2.304605e+07


In [31]:
clustered_tellco_exprience_df.to_csv('../data/user_experience_data.csv')

In [32]:
with open("../models/user_experience.pkl", "wb") as f:
    pickle.dump(kmeans, f)